In [1]:
# pip install pymorphy2 >> None

In [2]:
import pandas as pd
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# задаем директорию
file_path = '../datasets/'

In [28]:
# открываем файлы с данными
posts = pd.read_csv(os.path.join(file_path, 'posts.csv'))
profiles = pd.read_csv(os.path.join(file_path, 'profiles.csv'), index_col=0)

In [29]:
# открываем датафрейм с постами
posts.head(2)

,Unnamed: 0,user_id,text,likes,comments,reposts
0,0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [6]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [7]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [8]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [9]:
# удалим посты на украинском языке

# определяем шаблон для украинских символов (по специфичным для данного языка символам)
ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# создаем маску, указывающую строки, в которых столбец "post" содержит текст на украинском языке
mask = posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)

# сохраняем в датафрейме только строки, в которых маска имеет значение False
posts = posts[~mask]


# определяем шаблон для украинских символов
# ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# # сохраняем в датафрейме только те строки, в которых столбец "post" не содержит текста на украинском языке
# posts = posts[~posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)]

In [10]:
posts.head()

,user_id,post,likes,comments,reposts
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0
2,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0
3,ali-wodan,"Новый пост подкаста ""Миражи"" на всех платформах:В аудио: https://lnkd.in/giWuSVNВ инстаграме: https://lnkd.in/gbV6yxKН #automotive # # #cars #а патреоне: https://lnkd.in/gfShqTU#саморазвитие #психология #психологиячеловека #психологияжизни #психологияличности #личностныйрост #подкаст #подкастмиражи #маркетинг #осознанность #мысли #мысливслух #установки #влияние #подсознаниеИспользованные звуки, музыка и картинки:Аудиоцитата из фильма ""Формула любви"" Марка ЗахароваJardins du Luxembourg by Jahzzar is licensed under a Attribution-ShareAlike 3.0 International License.prisoner by Luis Prado from the Noun Project",1,0,0
4,ali-wodan,"Подкаст Миражи, Эпизод 13Ошибка невозвратных затратpodcast.ru/1539345144#подкаст #podcast #it #podcasts #psychology #психология #miragespodcast",2,0,0


In [11]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [12]:
# удаляем слова, которые идут после хэш-тэга
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'#[^\s]+', '', x))

In [13]:
# производим замену дефиса на пробел
posts["post_lemmatized"] = posts["post_lemmatized"].str.replace("-", " ")

In [14]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)

# только русские буквы и пробелы
# posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', '', regex=True) 

# только русские, английские буквы и пробелы
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'[^а-яА-ЯёЁa-zA-Z\s]', '', x))


# Шаблон '\W' в методе str.replace() представляет любой символ, не состоящий из слов, в т.ч. пробелы.
# Шаблон '[^\w\s]' пробелы оставляет, '[^\w\s\d]' + цифры

In [15]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
file_path_words = os.path.join(file_path, 'stop_words.txt')
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [17]:
'''если НЕ оставляем в постах английские слова'''
# # определяем шаблон регулярного выражения для русских букв
# pattern = '[^а-яА-ЯёЁ]'
# # создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
# mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
# # фильтруем датафрейм, используя маску
# posts = posts[mask]

'''если оставляем в постах английские слова'''
pattern = '[^а-яА-ЯёЁ]'
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
posts = posts[mask]

english_letters_mask = posts['post_lemmatized'].str.match(r'^[a-zA-Z\s]+$')
posts = posts[~english_letters_mask]

In [18]:
posts.sample(20)

,user_id,post,likes,comments,reposts,post_lemmatized
2649,kokot-dmytro,"I’m #hiring for an open role. Know anyone who would be a good fit? #dotnet В поисках #csharp талантов на bigdata Облака, кубер, clickhouse и многое другое",3,0,0,open role know anyone who good fit поиск талант bigdata облака кубер clickhouse многое
4070,marina-a-577a64232,"Часть 1 Insomnia (бессонница), или «Sweet dreams»#it #work #psychology #sweet #dream #love #libido #network #help #helpme #comment #online #blog #results #surfing #internet #time #money #insomnia #sleep #good #night #patternПоговорим про наш сон.Наше состояние утром зависит от того, во сколько мы легли и во сколько проснулись. Но еще очень важно, какие фазы сна мы прошли за ночь и в какую проснулся человек. Выделяют две основные фазы: медленного сна и быстрого сна. Медленный сон. В это время происходит полное расслабление мышц, замедляется сердечно-сосудистая деятельность и дыхание. С психологической точки зрения ИМЕННО в эту фазу сна происходит раскладывание информации «по полочкам» и ее запоминание, восстанавливаются нейронные связи и наше тело. Недаром говорят: утро вечера мудренее. Вслед за фазой медленного сна наступает фаза быстрого. В эту фазу происходит активация внутренних органов и систем, ускоряется сердцебиение, учащается дыхание, самое главное, происходит активация глазодвигательных мышц – начинают двигаться глаза. С психологической точки зрения эта фаза характеризуется сновидениями. Сны – это проработка головным мозгом неосознанного материала. За ночь фазы медленного и быстрого сна чередуются примерно четыре-шесть раз. Паттерн – фаза медленного и быстрого сна – как раз и длится примерно 1,5-2 часа. В первой половине ночи больше и длиннее фазы медленного сна. Во второй половине ночи фазы быстрого сна становятся длиннее. Если ложиться спать после 12 часов, то, сколько бы Вы не спали, все равно будете чувствовать усталость, потому что не было достаточно времени на переработку всей информации, поступившей за день. Поэтому хорошо было бы ложиться не позднее, чем за час до полуночи, лучше – за два. Если человек просыпается в фазу медленного сна, он чувствует себя усталым, будто не выспался. Если просыпается в фазу быстрого сна, чувствует себя бодро. Вообще сам организм диктует нам это время пробуждения. Если мы вовремя легли, ночь спали и не просыпались, то мы сами проснёмся в фазу быстрого сна.И кстати, спать «в запас» на выходных не получится, оно так не работает. Единственное, чем это закончится, Вы заснете еще позже в следующую ночь.Поэтому старайтесь выработать ежедневый режим сна и ложится не позже 23.00 часов.ПОВТОРЯЮ!!! Почему не стоит «серфить» или «листать» телефон перед сном. Синий цвет экрана, попадая на сетчатку глаза, тормозит выработку гормона мелатонина, который вызывает сон. А вот занятие «любовью» благодаря эндорфинам сделай сон лучше, а вечер не забываемым.",2,0,0,часть insomnia бессонница sweet dreams соннаша состояние утром зависеть сколько лечь сколько проснулись ещё очень важно фаза сон пройти ночь проснуться человек выделять основной фазы медленный сон быстрый сна медленный сон время происходить полный расслабление мышц замедляться сердечно сосудистый деятельность дыхание психологический точка зрение именно фаза сон происходить раскладывание информация полочкам запоминание восстанавливаться нейронный связь тело недаром говорят утро вечер мудренее вслед фаза медленный сон наступать фаза быстрого фаза происходить активация внутренний орган систем ускоряться сердцебиение учащаться дыхание самый главное происходить активация глазодвигательный мышца начинать двигаться глаза психологический точка зрение фаза характеризоваться сновидениями сон проработка головной мозг неосознанный материала ночь фаза медленный быстрый сон чередоваться примерно четыре шесть паттерн фаза медленный быстрый сон длиться примерно часа первый половина ночь большой длинный фаза медленный сна второй половина ночь фаза быстрый сон становиться длиннее ложиться спать часов сколько спали всё 

In [19]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2168 entries, 0 to 8597
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          2168 non-null   object
 1   post             2168 non-null   object
 2   likes            2168 non-null   object
 3   comments         2168 non-null   int64 
 4   reposts          2168 non-null   int64 
 5   post_lemmatized  2168 non-null   object
dtypes: int64(2), object(4)
memory usage: 118.6+ KB


Объединяем датафреймы

In [20]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [21]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [22]:
# удаляем лишние столбцы
#df = df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [23]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [24]:
# удаляем из столбца likes точки, запятые и пробелы
df["likes"] = df["likes"].replace(r'\.|\,|\s', '', regex=True)

# меняем тип данных сотлбца likes на integer
df["likes"] = df["likes"].astype("int64")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2156 entries, 0 to 2167
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2156 non-null   object
 1   post              2156 non-null   object
 2   likes             2156 non-null   int64 
 3   comments          2156 non-null   int64 
 4   reposts           2156 non-null   int64 
 5   post_lemmatized   2156 non-null   object
 6   user_name         2156 non-null   object
 7   user_head         2156 non-null   object
 8   user_work         2064 non-null   object
 9   user_position     2156 non-null   object
 10  user_tags         412 non-null    object
 11  user_location     2149 non-null   object
 12  user_viewers      2156 non-null   object
 13  user_contacts     2156 non-null   object
 14  user_common_info  1961 non-null   object
dtypes: int64(3), object(12)
memory usage: 269.5+ KB


In [26]:
df.sample(5)

,user_id,post,likes,comments,reposts,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
392,nataliagosteva,"Так как не ко всем из Juniors работа мечты сама постучится в двери, как к Карлу III, делюсь подборкой телеграмм-каналов с вакансиями для QA (и не только) и возможностью оставить своё резюме:1. https://t.me/Remoteit - вакансии и проекты на удалёнке.2. https://t.me/Gamedevjobs - gamedev вакансии.3. https://t.me/PBH_JOBS - вакансии и запросы на поиск работы в IT в Польше + живое общение.4. https://t.me/belarusitwant - вакансии, курсы, стажировки и митапы.5. https://t.me/forallqa - вакансии для QA и TestOps6. https://t.me/digital_hr - вакансии для IT и Digital7. https://t.me/qajoboffer - вакансии для QA8. https://t.me/jobforjunior - вакансии для начинающих специалистов в IT и Digital9. https://lnkd.in/d885w3c6 - вакансии и резюме it-специалистов в Польше10. https://t.me/qa_jobs - аналитика и вакансии для QA11. https://t.me/relocaty_jobs - вакансии с релокацией и резюме12. https://t.me/young_june - вакансии для начинающих it-специалистов в СНГ13. https://t.me/itlenta - вакансии и резюме14. https://t.me/evacuatejobs - вакансии с релокацией и удалёнкой15. https://t.me/belit_jobs - вакансии и резюме16. https://lnkd.in/dERGaNeg - резюме кандидатов в поиске 17. https://t.me/Cvflow - резюме соискателей 18. https://t.me/remotejun - вакансии для начинающих it-специалистов на удаленку19. https://t.me/relocateme - вакансии с релокацией20. https://lnkd.in/dWmkiFK3 - стажировка и вакансии для молодых специалистов21. https://t.me/Relocats - вакансии с релокацией22. https://t.me/jobsearchIT - вакансии и резюме23. https://t.me/jobGeeks - поток вакансий24. https://lnkd.in/dhQ3gnSa - вакансии с релокацией 25. https://t.me/remotejobss - вакансии на удалёнке26. https://t.me/qa_resumes - резюме специалистов QA27. https://lnkd.in/dcGRCMtQ - вакансии и резюмеСохраняйте, делитесь постом и добавляйте в комментарии полезные ресурсыВдруг, именно там у кого-то сойдутся звёзды!#juniors #qa #opentowork #lookingforjob #cv #vacancy #search",1209,30,294,juniors работа мечта постучаться двери карл iii делиться подборка телеграмм канал вакансия qa возможность оставить резюме httpstmeremoteit вакансия проект удалёнке httpstmegamedevjobs gamedev вакансии httpstmepbhjobs вакансия запрос поиск работа польша живой общение httpstmebelarusitwant вакансии курсы стажировка митапы httpstmeforallqa вакансия qa testops httpstmedigitalhr вакансия digital httpstmeqajoboffer вакансия qa httpstmejobforjunior вакансия начинающий специалист digital httpslnkdindwc вакансия резюме специалист польше httpstmeqajobs аналитика вакансия qa httpstmerelocatyjobs вакансия релокация резюме httpstmeyoungjune вакансия начинающий специалист снг httpstmeitlenta вакансия резюме httpstmeevacuatejobs вакансия релокация удалёнкой httpstmebelitjobs вакансия резюме httpslnkdinderganeg резюме кандидат поиск httpstmecvflow резюме соискатель httpstmeremotejun вакансия начинающий специалист удаленку httpstmerelocateme вакансия релокацией httpslnkdindwmkifk стажировка вакансия молодой специалистов httpstmerelocats вакансия релокацией httpstmejobsearchit вакансия резюме httpstmejobgeeks поток вакансий httpslnkdindhqgnsa вакансия релокация httpstmeremotejobss вакансия удалёнке httpstmeqaresumes резюме специалист qa httpslnkdindcgrcmtq вакансия резюмесохраняйте делиться пост добавлять комментарий полезный ресурсывдруг именно сойтись звёзды,Natalia Gosteva,🦄 UX/UI designer | Open for new projects,Innotech,UX/UI designer,NaN,Russia,652,500+,Hi! I am a UX/UI Designer with experience in graphic design and project management.Experience highlights:• Conducted UX research including questionnaires and deep interviews to validate hypotheses and understand user needs.• Proficient in creating JTBD and CJM.• Designed both mobile and web applications from scratch.• Developed prototypes to bring ideas to life.I am committed to delivering quali

In [27]:
# сохраняем датафрейм
df.to_csv(os.path.join(file_path, 'linkedin.csv'))